In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
model_name = '01'

In [ ]:
model_pth = '../results/rewards/gradient/sub-'+model_name+'/'+'rewards.npy'
model_rewards = np.load(model_pth)

In [ ]:
subject_rewards_1_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d1.csv'
subject_rewards_2_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d2.csv'
subject_rewards_3_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d3.csv'
all_subject_reward_pths = [subject_rewards_1_pth, subject_rewards_2_pth, subject_rewards_3_pth]

In [ ]:
subject_rewards = [pd.read_csv(pth, header=None).values for pth in all_subject_reward_pths]
subject_rewards = np.vstack(subject_rewards)

In [ ]:
from sklearn.feature_selection import mutual_info_regression


# Flatten the model_rewards array to match the shape of subject_rewards
MIs = np.zeros((model_rewards.shape[0]))
for idx, model_reward in enumerate(model_rewards):
    model_rewards_flat = model_reward.flatten()
    subject_rewards_flat = subject_rewards.flatten()

    # Calculate mutual information
    mutual_info = mutual_info_regression(subject_rewards_flat, model_rewards_flat)
    MIs[idx] = mutual_info

In [ ]:
plt.plot(MIs)